创建人工数据集，存储到csv（逗号）文件

这行代码是Python中的代码，它使用了os模块的makedirs函数来创建文件夹。

os.makedirs(os.path.join('.','data'), exist_ok=True)

os.makedirs：这个函数用于创建文件夹。如果文件夹已经存在，它会抛出一个错误。但是，通过设置exist_ok=True，如果文件夹已经存在，它不会抛出错误，而是继续执行。
os.path.join('.','data')：这个函数用于连接两个路径，这里是当前目录（.表示当前目录）和data文件夹名称。所以它创建的是在当前目录下的data文件夹。
所以，这行代码的意思是：在当前目录下创建名为"data"的文件夹，如果该文件夹已经存在，则不抛出错误，继续执行。

In [1]:
import os

os.makedirs(os.path.join('.','data'), exist_ok=True)

data_file = os.path.join('.', 'data', 'house_tiny.csv')

with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')
    f.write('NA,Pave,127500\n')
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

In [2]:
import pandas as pd

data = pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


为了处理缺失的数据，典型的方法包括 插值 和 删除，这里采用插值

.iloc[]基于整数位置选择数据

In [8]:
inputs, outputs = data.iloc[:,0:2], data.iloc[:,2]
inputs = inputs.fillna(inputs.mean())
print(inputs)

   NumRooms Alley
0       3.0  Pave
1       2.0   NaN
2       4.0   NaN
3       3.0   NaN


/tmp/ipykernel_806/926360299.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  inputs = inputs.fillna(inputs.mean())


In [9]:
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0       3.0           1          0
1       2.0           0          1
2       4.0           0          1
3       3.0           0          1


pd.get_dummies() 是 pandas 中的一个函数，用于进行独热编码 (One-Hot Encoding)。独热编码通常用于将分类变量（Categorical Variable）转换为一种格式，以便能够提供给机器学习算法。

NumRooms 列中的数值不会被改变，因为它是一个数值型变量，不是分类变量。
Alley 列中的值 "Pave" 会被转换为一个新的列，名为 Alley_Pave，并为该列赋值。如果原始数据中某行的 Alley 值为 "Pave"，则新列 Alley_Pave 中的对应行值为 1，否则为 0。
由于 dummy_na=True，任何NaN值（在你的数据中为 "NaN"）在 Alley 列中也会被编码为一个新的列。所以会出现一个名为 Alley_nan 的新列。如果原始数据中某行的 Alley 值为NaN，则新列 Alley_nan 中的对应行值为 1，否则为 0。
经过上述转换后，你的数据可能会变成这样：


现在 inputs，outputs 中所有条目都是数值类型，他可以转化为张量格式。

In [13]:
import torch
print("inputs.values:",inputs.values)
print("outputs.values:",outputs.values)
x,y = torch.tensor(inputs.values), torch.tensor(outputs.values)
x,y

inputs.values: [[3. 1. 0.]
 [2. 0. 1.]
 [4. 0. 1.]
 [3. 0. 1.]]
outputs.values: [127500 106000 178100 140000]


(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500, 106000, 178100, 140000]))